# Eigenvalue problem

### Initial tests:

Given the matrix A above:

In [427]:
import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
import math

A = np.array([[3, 4, 0],
              [4, 3, 0],
              [0, 0, 2]])

Solving the characteristic equation `det(A - λI) = 0` we get the following eigenvalues:

In [428]:
lambda1 = -1
lambda2 = 2
lambda3 = 7

Then, replacing those eigenvalues in `Ax = λx` we get the following normalized associated eigenvectors:

In [641]:
# associated with lambda1 = -1
x1 = np.array([0.707,
              -0.707,
               0.])


# associated with lambda2 = 2
x2 = np.array([0.243,
              -0.970,
               0.])

# associated with lambda3 = 7
x3 = np.array([0.707,
               0.707,
               0.])


Now that we have the exact solution, we'll use the QR method to find an approximate solution by leveraging the Householder transformation:

In [616]:
def multiply_by_scalar(matrix, scalar):
    result = np.zeros(shape=dimension_of(matrix))
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            result[i][j] = scalar * matrix[i][j]
    return result    

def scalar_product(a, b):
    result = 0
    for i in range(len(a[0])):
        result += a[0][i]*b[0][i]
    return result

def multiply_matrixes(a, b):
    m,n = dimension_of(a)
    _,p = dimension_of(b)
    result = np.zeros(shape=(m,p))
    for i in range(m):
        for j in range(p):
            for k in range(n):
                result[i][j] += a[i][k] * b[k][j]
    return result                          

def nth_identity(n):
    return np.identity(n)

def sum_matrixes(a, b):
    result = np.zeros(shape=dimension_of(a))
    for i in range(len(a)):
        for j in range(len(a[0])):
            result[i][j] += a[i][j] + b[i][j]
    return result
    
def dimension_of(matrix):
    return matrix.shape

def nth_canonical_form(n, dimension):
    zeros = np.zeros((1,dimension[1]))
    zeros[0][n-1] = 1
    return zeros

def norm(v):
    sum_square = 0
    for i in v[0]:
        sum_square += math.pow(i, 2)
    return math.sqrt(sum_square)

def compute_sigma(a): # by convention we're using the signal of first element
    if a[0][0] >= 0:
        return 1
    return -1

def compute_vi(A, i): # computes vi for finding the householder matrix
    ai = np.array([A[:, i - 1]])
    ei = nth_canonical_form(i, dimension_of(A))
    sigma = compute_sigma(ai)
    return sum_matrixes(ai, multiply_by_scalar(ei, sigma*(norm(ai))))

def compute_hi(A, i): # computes householder matrix
    vi = compute_vi(A, i)
    scalar = -2/scalar_product(vi,vi)
    matrix = multiply_matrixes(vi.transpose(), vi)
    identity = nth_identity(dimension_of(A)[0])
    return sum_matrixes(identity, multiply_by_scalar(matrix, scalar))

def compute_h_list(A):
    Ai = np.copy(A)
    result = []
    for i in range(1, len(A)):
        Hi = compute_hi(Ai, i)
        result.append(Hi)
        Ai = multiply_matrixes(Hi, Ai)
    return result

def compute_qi(H_list):
    Q = nth_identity(len(H_list[0]))
    for i in range(len(H_list)):
        Q = multiply_matrixes(Q, H_list[i])
    return Q
        
def compute_ri(H_list, A):        
    R = nth_identity(len(A))
    for i in range(len(H_list)):
        R = multiply_matrixes(R, H_list[len(H_list) - i -1])        
    return multiply_matrixes(R, A)

def qr_factorization(A, iterations):
    Ai = np.copy(A)
    Qi = nth_identity(dimension_of(A)[0])
    Vi = np.copy(Qi)
    for i in range(1,iterations+1):
        H_list = compute_h_list(Ai)       
        Ri = compute_ri(H_list, Ai)
        Qi = compute_qi(H_list)
        Vi = multiply_matrixes(Vi, Qi)
        Ai = multiply_matrixes(Ri, Qi)
    return Ai, Vi
    

In [642]:
result = qr_factorization(A, 200)
print(result[0]) # matrix where the main diagonal elements are the eigenvalues
print(result[1]) # matrix where the columns are the eigenvectors (if A is symetric)

[[-1.  0.  0.]
 [ 0.  7.  0.]
 [ 0.  0.  2.]]
[[-0.707 -0.707  0.   ]
 [ 0.707 -0.707  0.   ]
 [ 0.     0.     1.   ]]
